# 基于路由的多数据源 RAG

理解可能有误

router retriver 是针对同一个数据源，采取不同的索引策略？

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 3.43 s, sys: 425 ms, total: 3.85 s
Wall time: 3.47 s


## 创建多个检索器

### 旅游信息检索器

In [2]:
%%time

items=[
    "颐和园",
    "北海公园",
    "故宫",
]

from llama_index.readers.web import TrafilaturaWebReader

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 740 ms, sys: 12.2 ms, total: 752 ms
Wall time: 984 ms


3

In [3]:
%%time

from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever

vector_index = VectorStoreIndex.from_documents(documents)
# attractions_retriever=vector_index.as_retriever()
attractions_retriever = VectorIndexRetriever(
    index=vector_index,
    # similarity_top_k=1,
)

CPU times: user 2.54 s, sys: 96.3 ms, total: 2.64 s
Wall time: 54.7 s


In [4]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
    retriever=attractions_retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query("北海公园在哪里？")
response.response

CPU times: user 107 ms, sys: 3.72 ms, total: 111 ms
Wall time: 2.28 s


'北海公园位于北京市西城区文津街1号。'

### 动物信息检索器

In [5]:
%%time

items=[
    "大熊猫",
    "东北虎",
    "狸花猫",
]

from llama_index.readers.web import TrafilaturaWebReader

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 302 ms, sys: 3.04 ms, total: 305 ms
Wall time: 551 ms


3

In [6]:
%%time

vector_index = VectorStoreIndex.from_documents(documents)
# animals_retriever=vector_index.as_retriever()
animals_retriever = VectorIndexRetriever(
    index=vector_index,
    # similarity_top_k=1,
)

CPU times: user 1.84 s, sys: 72.1 ms, total: 1.91 s
Wall time: 38.8 s


In [7]:
%%time

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
    retriever=animals_retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query("东北虎有多重？")
response.response

CPU times: user 22 ms, sys: 3.36 ms, total: 25.4 ms
Wall time: 626 ms


'成年东北虎雄性的平均体重为250千克。'

## 实现路由查询

In [8]:
%%time

from llama_index.core.tools import RetrieverTool

attractions_tool = RetrieverTool.from_defaults(
    retriever=attractions_retriever,
    description=(
        "检索有关旅游景点方面信息上下文"
        "目前包括： 颐和园、北海公园和故宫等"
    ),
)

CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 14.3 µs


In [9]:
%%time

animals_tool = RetrieverTool.from_defaults(
    retriever=animals_retriever,
    description=(
        "检索有关动物方面信息上下文"
    ),
)

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 10.3 µs


In [13]:
%%time

from llama_index.core.retrievers import RouterRetriever
from llama_index.core.selectors import LLMSingleSelector
# from llama_index.selectors.llm_selectors import LLMSingleSelector
from llama_index.llms.openai import OpenAI

retriever = RouterRetriever(
    selector=LLMSingleSelector.from_defaults(),
    retriever_tools=[
        attractions_retriever,
        animals_retriever,
    ],
)

AttributeError: 'VectorIndexRetriever' object has no attribute 'retriever'